In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


## Setup training/test data

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train_l), (x_test, y_test_l) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_l, num_classes)
y_test = keras.utils.to_categorical(y_test_l, num_classes)

## Load pretrained model

In [ ]:
model_save_name = 'mnist_classifier'
path = F"{model_save_name}" 
#model.save(path)
model = keras.models.load_model(path)
model.layer_summary()

## Compute network layer activations for each image

In [40]:
%reload_ext autoreload
%autoreload 2

import nap
ap = nap.NeuralActivationPattern(x_test[:1000], y_test_l[:1000], model)

In [41]:
def show_image_grid(images, labels, img_idx, title, images_per_row, img_scale):
    import plotly.express as px
    imSize = (images[0].shape[0], images[0].shape[1])
    # Get indices of the largest 
    sampleSize = len(img_idx)
    nCols = min(sampleSize, images_per_row)
    nRows =  1 +sampleSize % images_per_row
    img = np.array([images[idx].reshape(imSize) for idx in img_idx])
    fig = px.imshow(img, facet_col=0, binary_string=True, facet_col_wrap=nCols, 
                    #width=int(nCols*imSize[0]*img_scale), 
                    #height=50+int(nRows*10 + nRows*imSize[1]*img_scale), 
                    title=title
                    )
    # Set facet titles
    for i, im in enumerate(img_idx):
        fig.layout.annotations[i]['text'] = f"{labels[im]}"
    #fig.update_yaxes(automargin=False, title=title)
    #fig.update_xaxes(automargin=True)
    fig.update_layout(margin=dict(b=0))
    #fig.update_annotations(standoff=10)
    fig.show()

def show_pattern(average, representatives, outliers, images, labels, title):
    import plotly.express as px
    import plotly.subplots as sp
    imSize = (images[0].shape[0], images[0].shape[1])
    img = np.array([average.reshape(imSize)] + [images[idx].reshape(imSize) for idx in representatives] + [images[idx].reshape(imSize) for idx in outliers])
    fig = px.imshow(img, facet_col=0, binary_string=True, title = title)
    fig.layout.annotations[0]['text'] = "Average"
    for i, im in enumerate(representatives):
        fig.layout.annotations[1+i]['text'] = f"Representative | {labels[im]}"
    for i, im in enumerate(outliers):
        fig.layout.annotations[len(representatives)+1+i]['text'] = f"Outlier | {labels[im]}"
    fig.show()

def show_images(images, labels, layer_img_idx, titles, images_per_row = 10, img_scale = 7.0):
    for layer, img_idx in enumerate(layer_img_idx):
        show_image_grid(images, labels, img_idx, titles[layer], images_per_row, img_scale)


## Show the images generating highest activations for each layer

In [42]:
max_activations = [ap.layer_max_activations(layerId, agg_func = np.average) for layerId in range(len(model.layers))]
titles = [f"Highest activating images for layer {layerId}" for layerId in range(len(max_activations))]
show_images(x_test, y_test_l, max_activations, titles)

             Layer    Activation shape
0           conv2d  (1000, 26, 26, 32)
1    max_pooling2d  (1000, 13, 13, 32)
2         conv2d_1  (1000, 11, 11, 64)
3  max_pooling2d_1    (1000, 5, 5, 64)
4          flatten        (1000, 1600)
5          dropout        (1000, 1600)
6            dense          (1000, 10)


# Neural Activation Patterns

In [92]:
nSamplesPerLayer = 10
layerId = 3

### Show overview of patterns with respect to output data (y) 

In [101]:
%reload_ext autoreload
%autoreload 2

import nap
ap.layer_summary(layerId)

                 counts  counts_norm
patternId label                     
-1.0      0          57     0.487179
          1           9     0.076923
          2          99     0.846154
          3          89     0.760684
          4         101     0.863248
          5          87     0.743590
          6          75     0.641026
          7          85     0.726496
          8          77     0.658120
          9          78     0.666667
 0.0      2           9     0.076923
 1.0      2           7     0.059829
 2.0      0          28     0.239316
          2           1     0.008547
 3.0      3          18     0.153846
 4.0      8          12     0.102564
 5.0      6           7     0.059829
 6.0      6           5     0.042735
 7.0      4           9     0.076923
 8.0      9           6     0.051282
 9.0      1         117     1.000000
          7           1     0.008547
 10.0     9           5     0.042735
 11.0     7          11     0.094017
 12.0     7           2     0.017094
 

## Show inputs treated the same way by the network

In [ ]:
sorted_patterns = ap.sorted(layerId)

for pattern_id, pattern in sorted_patterns.groupby('patternId'):
    avg = ap.average(pattern.index)
    centers = pattern.head(1).index
    outliers = pattern.tail(3).index
    show_pattern(avg, centers, outliers, x_test, y_test_l, F"Layer {layerId}, Pattern: {pattern_id}, Size: {len(pattern)}")

In [ ]:
layerId = 3
filterId = 1
sorted_patterns = nap.sort(ap.filter_patterns(layerId, filterId))

for pattern_id, pattern in sorted_patterns.groupby('patternId'):
    avg = ap.average(pattern.index)
    centers = pattern.head(1).index
    outliers = pattern.tail(3).index
    show_pattern(avg, centers, outliers, x_test, y_test_l, F"Layer {layerId}, Filter: {filterId}, Pattern: {pattern_id}, Size: {len(pattern)}")

In [ ]:

frac = 0.1
pattern_samples = ap.head(layerId, nSamplesPerLayer) 
titles = []
patterns = []
for pattern_id, pattern in pattern_samples.groupby('patternId'):
    patterns.append(pattern.index)
    titles.append(F"Layer {layerId}, Pattern: {pattern_id} - Images most likely belonging to pattern")
show_images(x_test, y_test_l, patterns, titles)

### Show aggregated inputs treated the same way 

In [ ]:
for layerId, layer in enumerate(model.layers):
    averages = []
    pattern_labels = []
    for pattern_id, pattern in ap.layer_patterns(layerId).groupby('patternId'):
        averages.append(ap.average(pattern.index))
        pattern_labels.append(f"Id: {pattern_id}, Size: {len(pattern)}")
    nPatterns = len(averages)
    indices = list(range(nPatterns))
    show_images(averages, pattern_labels, [indices], [f"Layer {layerId} - Pattern averages"])


### Show activation for each feature. Warning!!! slow code. 

In [ ]:

# Convert to slow, but plotting friendly pandas DataFrame
activations_df = {'input_data': [], 'layer': [], 'feature': [], 'activation':[]}
for dp in range(len(layer_activations)):
  for feature in range(len(ap.layer_activations[0])):
      for activation in layer_activations[:][feature]:
        activations_df['input_data'].append(dp)
        activations_df['layer'].append(patternOnLayer)
        activations_df['feature'].append(feature)
        activations_df['activation'].append(activation)

activations_df = pd.DataFrame(activations_df)
activations_df.describe()

In [ ]:
px.scatter(activations_df, x='feature', y='activation')